# Question 1
load data is_sleepy.csv to a dataframe

(a) what is the entropy of 'meal' 

In [74]:
import pandas as pd
from scipy.stats import entropy
import math

pd = pd.read_csv("is_sleepy.csv")

def log2(x):
    return math.log(x,2)

def custom_entropy(col):
    p = len(col.where(col == 1).dropna())/len(col)
    return -1*p*log2(p) - ((1 - p)* log2(1 - p))

def conditional_entropy(col1, col2):
    a = custom_entropy(col1[col2 == 1]) * probability_col(col2)
    b = custom_entropy(col1[col2 == 0]) * (1-probability_col(col2))
    return a + b

def probability_col(col):
    return len(col.where(col == 1).dropna())/len(col)

def probability_col_given(col1, col2):
    col = col1.where(col2 == 1).dropna()
    return len(col.where(col == 1).dropna())/len(col)

def joint_dist(col1, col2, df):
    col = col1.where(col2 == 1).dropna()
    return len(col.where(col == 1).dropna())/len(df)


meal_entropy = custom_entropy(pd.meal)
print(meal_entropy)


0.9426831892554922


(b) what is the entropy of 'tired'

In [75]:


tired_entropy = custom_entropy(pd.tired)
print(tired_entropy)

0.904381457724494


(c) what is the conditional entropy of 'meal' given 'tired'

In [76]:
meal_given_tired = pd.meal.where(pd.tired == 1).dropna()
meal_given_tired_entropy = custom_entropy(meal_given_tired)
print(meal_given_tired_entropy)

0.7871265862012691


(d) look at the definition of mutual information from the lecture and answer

$$ I(X;Y) \overset{?}{=} I(Y;X) $$

(e) compute 
$$ I(X;Y)=H(X)-H(X\mid Y) $$
and 
$$ I(Y;X)=H(Y)-H(Y\mid X) $$
for X=meal and Y=tired
are they equal? do they satisfy 
$$ \left\|I(Y;X)-I(X;Y)\right\|<1\cdot 10^{-15} $$

In [81]:

x = pd.meal
y = pd.tired

hxy = conditional_entropy(x,y)
hyx = conditional_entropy(y,x)
ixy = custom_entropy(x) - hxy
iyx = custom_entropy(y) - hyx


if abs(iyx - ixy) < 10**(-15):
    print("I(Y;X) = I(X;Y) are equal")
else:
    print("I(Y;X) = I(X;Y) are not equal")

I(Y;X) = I(X;Y) are equal


# Question 2
use the same data from question 1. our target is to predict sleepy column using the other columns

(a) what is the optimal question for the data $ qa $ using the information gain criterion?

Optimal question is asking about tiredness

In [90]:
print("Insight about tired from sleepy:", custom_entropy(pd.sleepy) - conditional_entropy(pd.sleepy, pd.tired))
print("Insight about meal from sleepy:", custom_entropy(pd.sleepy) - conditional_entropy(pd.sleepy, pd.meal))
print("Insight about exercise from sleepy:", custom_entropy(pd.sleepy) - conditional_entropy(pd.sleepy, pd.exercise))

Insight about tired from sleepy: 0.1550221871551789
Insight about meal from sleepy: 0.03410217068156207
Insight about exercise from sleepy: 0.0017863578091161791


(b) we segment the data as following

\begin{array}{|c|c|c|}
\hline
\boldsymbol{math} & \boldsymbol{python} & \textbf{in words} \\
\hline
\hline
\hline
D_{1y} & D.qa==1 & qa\text{ is yes}\\
\hline
D_{1n} & D.qa==0 & qa\text{ is no}\\
\hline
\end{array}


you can now ask another question $ qb $
you can ask it regarding $ D_{1y} $ or $ D_{1n} $

What is the best question for $ D_{1y} $? What is the best question for $ D_{1n} $?

Whether the answer is yes or no we get more insight from asking if he is exercising than asking if he had a meal

In [96]:
dy = pd[pd.tired == 1]
dn = pd[pd.tired == 0]

print("Insight about meal from sleepy given the person is tired:", custom_entropy(dy.sleepy) - conditional_entropy(dy.sleepy, dy.meal))
print("Insight about exercise from sleepy given the person is tired:", custom_entropy(dy.sleepy) - conditional_entropy(dy.sleepy, dy.exercise))

print("Insight about meal from sleepy given the person is not tired:", custom_entropy(dn.sleepy) - conditional_entropy(dn.sleepy, dn.meal))
print("Insight about exercise from sleepy given the person is tired:", custom_entropy(dn.sleepy) - conditional_entropy(dn.sleepy, dn.exercise))

Insight about meal from sleepy given the person is tired: 0.00011420613134072166
Insight about exercise from sleepy given the person is tired: 0.053981378495651766
Insight about meal from sleepy given the person is not tired: 0.006855286754329559
Insight about exercise from sleepy given the person is tired: 0.10584334459644862


**Bonus**

(c) if you can only ask the question $ q2 $ for either $ D_{1y} $ or $ D_{1n} $, which one should you choose?

I would answer D1n becuase it gives us more insight that D1y